# W4-CRISPRi-workflow

Transcriptional interference is another powerful tool for assessing the function of genes, by expressing a guide RNA that targets a dead-Cas9/sgRNA complex to bind to the + (Watson) strand of the promoter and 5’-UTR of the targeted gene, thereby sterically blocking binding and progression of RNA polymerases to the promoter region. Workflow 4 uses CRISPR interference (CRISPRi) with ssDNA bridging to reversibly inactivate genes transcriptionally12,29 (Figure 5A). This approach targets regions upstream of the Transcriptional Start Site (TSS), using a dCas9-sgRNA complex positioned near the TSS (default 100bp upstream) to sterically hinder transcription. This allows for the functional study of genes through controlled knockdown. To get started, users can download the pCRISPR-dCas9 plasmid and the S. coelicolor (A3) genome. StreptoAIM will then generate all necessary components, including primers and plasmids.

In [13]:
import sys
import os
from pydna.dseqrecord import Dseqrecord
from datetime import datetime


# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from streptocad.sequence_loading.sequence_loading import (
    load_and_process_gene_sequences,
    load_and_process_genome_sequences,
    load_and_process_plasmid, 
    check_and_convert_input,
    annotate_dseqrecord,
    process_specified_gene_sequences_from_record)


from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging, make_ssDNA_oligos
from streptocad.crispr.guideRNA_crispri import extract_sgRNAs_for_crispri, SgRNAargs
from streptocad.primers.primer_generation import  primers_to_IDT
from streptocad.utils import ProjectDirectory,extract_metadata_to_dataframe

## INPUT

In [14]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../data/plasmids/pCRISPR–Cas9_plasmid_addgene.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
genes_to_KO = ['SCO5087', 'SCO5089', 'SCO5090']
genes_to_KO = ['80000-100000', '4000-7000', '9000-14000','15000-20000']


#### Advanced settings ####
# 4 Filtering metrics for sgRNAs
gc_upper = 0.72
gc_lower = 0.2
off_target_seed = 13
off_target_upper = 0
cas_type='cas9'
number_of_sgRNAs_per_group = 5
extension_to_promoter_region=200

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('CGGTTGGTAGGATCGACGGC')
dw_homology = Dseqrecord('GTTTTAGAGCTAGAAATAGC')

In [15]:
print(clean_plasmid.id)

pCRISPR-Cas9


# Computation

In [3]:
target_dict, genes_to_KO, annotation_input = check_and_convert_input(genes_to_KO)

print(annotation_input)
if annotation_input == True:
    genome = annotate_dseqrecord(genome, target_dict)


len(genome.features)


True
this is the target dict [{'region_1_(80000-100000)': [80000, 100000]}, {'region_2_(4000-7000)': [4000, 7000]}, {'region_3_(9000-14000)': [9000, 14000]}, {'region_4_(15000-20000)': [15000, 20000]}]
region_1_(80000-100000) [80000, 100000]
region_2_(4000-7000) [4000, 7000]
region_3_(9000-14000) [9000, 14000]
region_4_(15000-20000) [15000, 20000]


25828

In [4]:

# Initialize SgRNAArgs with desired parameters
args = SgRNAargs(genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9',
                extension_to_promoter_region=200,
                target_non_template_strand = True
                  )


In [5]:

sgrna_df = extract_sgRNAs_for_crispri(args)
sgrna_df

sgRNA or PAM generated were outside the designated border in region_1_(80000-100000). Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in region_1_(80000-100000). Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in region_1_(80000-100000). Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in region_1_(80000-100000). Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in region_1_(80000-100000). Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in region_1_(80000-100000). Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in region_1_(80000-100000). Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in region_1_(80000-100000). Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in region_2_(4000-7000). Skipp

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
8,NC_003888.3,region_1_(80000-100000),80001,1,-1,-22,0.70,CGG,CCCAGCGGTTGACCAAGCCG,GTTGACCAAGCCG,0
9,NC_003888.3,region_1_(80000-100000),80001,1,-1,-7,0.65,CGG,GGGAGAACGGCATCACCCAG,CGGCATCACCCAG,0
10,NC_003888.3,region_1_(80000-100000),80001,1,-1,6,0.60,CGG,GAGCGTATCTGCCGGGAGAA,TCTGCCGGGAGAA,0
11,NC_003888.3,region_1_(80000-100000),80001,1,-1,13,0.60,GGG,GCTCTTCGAGCGTATCTGCC,GAGCGTATCTGCC,0
12,NC_003888.3,region_1_(80000-100000),80001,1,-1,14,0.55,CGG,TGCTCTTCGAGCGTATCTGC,CGAGCGTATCTGC,0
15,NC_003888.3,region_1_(80000-100000),80001,1,-1,53,0.65,CGG,AGTTCACCGGCCGGCACCAA,CGGCCGGCACCAA,0
17,NC_003888.3,region_1_(80000-100000),80001,1,-1,66,0.55,CGG,GACAACGGCAAGCAGTTCAC,GCAAGCAGTTCAC,0
18,NC_003888.3,region_1_(80000-100000),80001,1,-1,81,0.50,CGG,TTCGAAGTGCTGACGGACAA,TGCTGACGGACAA,0
19,NC_003888.3,region_1_(80000-100000),80001,1,-1,88,0.60,CGG,CGTGCCCTTCGAAGTGCTGA,TTCGAAGTGCTGA,0


In [6]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
8,NC_003888.3,region_1_(80000-100000),80001,1,-1,-22,0.70,CGG,CCCAGCGGTTGACCAAGCCG,GTTGACCAAGCCG,0
9,NC_003888.3,region_1_(80000-100000),80001,1,-1,-7,0.65,CGG,GGGAGAACGGCATCACCCAG,CGGCATCACCCAG,0
10,NC_003888.3,region_1_(80000-100000),80001,1,-1,6,0.60,CGG,GAGCGTATCTGCCGGGAGAA,TCTGCCGGGAGAA,0
11,NC_003888.3,region_1_(80000-100000),80001,1,-1,13,0.60,GGG,GCTCTTCGAGCGTATCTGCC,GAGCGTATCTGCC,0
12,NC_003888.3,region_1_(80000-100000),80001,1,-1,14,0.55,CGG,TGCTCTTCGAGCGTATCTGC,CGAGCGTATCTGC,0


## Output

In [7]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)
print(list_of_ssDNAs[0].name)

# cut plasmid
from Bio.Restriction import NcoI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI), key=lambda x: len(x), reverse=True)[0]
#print(linearized_plasmid)

sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)
sgRNA_vectors

region_1_(80000-100000)_loc_-22


[Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279)]

In [8]:

# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"CRISPRi_{row['locus_tag']}_p{row['sgrna_loc']}"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'p{targeting_info[i]}_#{i+1}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  # Using the same value for ID as for name for simplicity
    sgRNA_vectors[i].description = f'Assembled plasmid targeting {", ".join(genes_to_KO)} for single gene KNOCK-DOWN, assembled using StreptoCAD.'


In [9]:
print_plasmids = False

if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR–dCas9/{vector.id}.gb")

In [10]:
integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
plasmid_metadata_df = extract_metadata_to_dataframe(sgRNA_vectors,
                                                    clean_plasmid,
                                                    integration_names)

plasmid_metadata_df

,plasmid_name,date,original_plasmid,integration,size
0,pCRISPRi_region_1_(80000-100000)_p-22_#1,2024-07-23,.,sgRNA_region_1_(80000-100000)(-22),11279
1,pCRISPRi_region_1_(80000-100000)_p-7_#2,2024-07-23,.,sgRNA_region_1_(80000-100000)(-7),11279
2,pCRISPRi_region_1_(80000-100000)_p6_#3,2024-07-23,.,sgRNA_region_1_(80000-100000)(6),11279
3,pCRISPRi_region_1_(80000-100000)_p13_#4,2024-07-23,.,sgRNA_region_1_(80000-100000)(13),11279
4,pCRISPRi_region_1_(80000-100000)_p14_#5,2024-07-23,.,sgRNA_region_1_(80000-100000)(14),11279


### IDT primers

In [11]:
idt_primers=primers_to_IDT(list_of_ssDNAs)
idt_primers

,Name,Sequence,Concentration,Purification
0,region_1_(80000-100000)_loc_-22,CGGTTGGTAGGATCGACGGCCCCAGCGGTTGACCAAGCCGGTTTTA...,25nm,STD
1,region_1_(80000-100000)_loc_-7,CGGTTGGTAGGATCGACGGCGGGAGAACGGCATCACCCAGGTTTTA...,25nm,STD
2,region_1_(80000-100000)_loc_6,CGGTTGGTAGGATCGACGGCGAGCGTATCTGCCGGGAGAAGTTTTA...,25nm,STD
3,region_1_(80000-100000)_loc_13,CGGTTGGTAGGATCGACGGCGCTCTTCGAGCGTATCTGCCGTTTTA...,25nm,STD
4,region_1_(80000-100000)_loc_14,CGGTTGGTAGGATCGACGGCTGCTCTTCGAGCGTATCTGCGTTTTA...,25nm,STD


## Folder with all the generated I/O

In [12]:
generate_data_folder = True 

if generate_data_folder:
    input_files = [
        {"name": "input_genome.gb", "content": genome},
        {"name": "input_plasmid.gb", "content": clean_plasmid}
    ]

    output_files = [
        {"name": "cBEST_w_sgRNAs.gb", "content": sgRNA_vectors}, # LIST OF Dseqrecords
        {"name": "full_idt.csv", "content": idt_primers},
        {"name": "sgrna_df.csv", "content": sgrna_df},
        {"name": "filtered_df.csv", "content": filtered_df},
        {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},
    ]

    input_values = {
        "genes_to_knockout": genes_to_KO,


        "filtering_metrics": {
            "gc_upper": gc_upper,
            "gc_lower": gc_lower,
            "off_target_seed": off_target_seed,
            "off_target_upper": off_target_upper,
            "cas_type": cas_type,
            "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group,
            'extension_to_promoter_region':extension_to_promoter_region,

        },
        "overlapping_sequences": {
            "up_homology": str(up_homology),
            "dw_homology": str(dw_homology)
        }
    }


    # Paths to Markdown files
    markdown_file_paths = [
        "../../protocols/conjugation_protcol.md",
        "../../protocols/single_target_crispr_plasmid_protcol.md"

    ]



    # Data and time
    timestamp = datetime.utcnow().isoformat()

    # Create project directory structure
    project_directory = ProjectDirectory(
        project_name=f"CRISPRi_workflow_{timestamp}",
        input_files=input_files,
        output_files=output_files,
        input_values=input_values,
        markdown_file_paths=markdown_file_paths
    )


    # DO You want to save the folder? 
    save_zip_folder = False 

    if save_zip_folder: 
        # Generate the project directory structure and get the zip content
        zip_content = project_directory.create_directory_structure(create_directories=False)

        # Save the zip file to disk (optional)
        with open("project_structure.zip", "wb") as f:
            f.write(zip_content)